In [1]:
import pandas as pd
import numpy as np
import os
os.chdir(os.path.dirname(os.getcwd()))
print(os.getcwd())

%load_ext autoreload
%autoreload 2
%reload_ext autoreload


from resources.constants import *

pictures_df = pd.read_csv(PICTURE_TRIPLETS_CSV_PATH, sep=CSV_SEPARATOR)
outfits_df = pd.read_csv(OUTFITS_CSV_PATH, sep=CSV_SEPARATOR)
user_triplets_df = pd.read_csv(USER_ACTIVITY_TRIPLETS_CSV_PATH, sep=CSV_SEPARATOR)
# Ensure tags are lists
outfits_df["tag_categories"] = outfits_df["tag_categories"].apply(eval)
outfits_df["outfit_tags"] = outfits_df["outfit_tags"].apply(eval)


c:\Users\kaborg15\Python_projects\Vibrent_Dataset_Collection


In [ ]:
import src.load_baseline_resources
import pickle
from resources.constants import EMBEDDING_MODEL_DICT_PICKLE_PATH
# loaded_embeddings_dict = src.load_baseline_resources.load_embeddings_form_folder()
# pickle.dump(loaded_embeddings_dict, open(EMBEDDING_MODEL_DICT_PICKLE_PATH, "wb"))

loaded_embeddings_dict = pickle.load(open(EMBEDDING_MODEL_DICT_PICKLE_PATH, "rb"))
pictures_df["embeddings"] = pictures_df["picture.id"].map(loaded_embeddings_dict)
outfit_pictures_df = pictures_df.groupby("outfit.id").agg({"picture.id": list, "embeddings": list}).reset_index()
outfits_df["embeddings"] = outfits_df["id"].map(outfit_pictures_df.set_index("outfit.id")["embeddings"])
na_embedding_outfit_ids = outfits_df[outfits_df["embeddings"].isna()]["id"]
outfits_df = outfits_df.dropna(subset=["embeddings"])

In [ ]:
# Introduce group to rental triplets
id_group_dict = outfits_df[["id", "group"]].to_dict(orient="records")
id_group_dict = {x["id"]: x["group"] for x in id_group_dict}
user_triplets_df["group"] = user_triplets_df["outfit.id"].map(id_group_dict)
# Remove triplets with no embeddings
user_triplets_df = user_triplets_df[~user_triplets_df["outfit.id"].isin(na_embedding_outfit_ids)]

In [ ]:
user_orders_df = user_triplets_df.groupby("customer.id").agg({"outfit.id": list, "group":list, "meta.validFrom":list, "derived.bookingTime":list}).reset_index()
user_orders_df["num_orders"] = user_orders_df["outfit.id"].apply(lambda x: len(x))
user_orders_df = user_orders_df[user_orders_df["num_orders"] > 1]
user_orders_df["num_orders"].describe()

count    2216.000000
mean       28.939982
std        41.905801
min         2.000000
25%         5.000000
50%        13.000000
75%        35.000000
max       356.000000
Name: num_orders, dtype: float64

In [13]:
import numpy as np

def leave_one_out_split(outfit_ids, groups, derived_booking_times):
    outfit_ids = np.array(outfit_ids)
    groups = np.array(groups)
    derived_booking_times = np.array(derived_booking_times)
    sorted_indices = np.argsort(derived_booking_times)
    return outfit_ids[sorted_indices[:-1]], outfit_ids[sorted_indices[-1]], groups[sorted_indices[:-1]], groups[sorted_indices[-1]], derived_booking_times[sorted_indices[:-1]], derived_booking_times[sorted_indices[-1]]

def leave_one_out_split_unique(outfit_ids, groups, derived_booking_times):
    outfit_ids = np.array(outfit_ids)
    groups = np.array(groups)
    derived_booking_times = np.array(derived_booking_times)
    
    sorted_indices = np.argsort(derived_booking_times)
    sorted_outfit_ids = outfit_ids[sorted_indices]
    sorted_groups = groups[sorted_indices]
    sorted_booking_times = derived_booking_times[sorted_indices]
    
    unique_groups, counts = np.unique(sorted_groups, return_counts=True)
    
    single_count_indices = np.where(counts == 1)[0]
    if len(single_count_indices) == 0:
        print(f"No unique outfit found with groups {groups}")
        return None
    
    unique_group = unique_groups[single_count_indices[0]]
    unique_group_index = np.where(sorted_groups == unique_group)[0][0]
    remaining_indices = np.arange(len(sorted_groups)) != unique_group_index
    
    return (
        sorted_outfit_ids[remaining_indices], sorted_outfit_ids[unique_group_index],
        sorted_groups[remaining_indices], sorted_groups[unique_group_index],
        sorted_booking_times[remaining_indices], sorted_booking_times[unique_group_index]
    )


user_splits = user_orders_df.apply(lambda x: leave_one_out_split(x["outfit.id"], x["group"], x["derived.bookingTime"]), axis=1)
user_splits_df = pd.DataFrame(user_splits.tolist(), columns=["train_outfit_ids", "test_outfit_id", "train_group", "test_group", "train_booking_times", "test_booking_time"])
user_splits_unique = user_orders_df.apply(lambda x: leave_one_out_split_unique(x["outfit.id"], x["group"], x["derived.bookingTime"]), axis=1)
user_splits_unique_df = pd.DataFrame(user_splits_unique.tolist(), columns=["train_outfit_ids", "test_outfit_id", "train_group", "test_group", "train_booking_times", "test_booking_time"])
user_splits_unique_df = user_splits_unique_df.dropna()

No unique outfit found with groups ['group.8abe6af9eccc8b578c2ef59628f8b454'
 'group.96f4cce22d4a236e0652c67fc9b18d12'
 'group.8abe6af9eccc8b578c2ef59628f8b454'
 'group.96f4cce22d4a236e0652c67fc9b18d12']


In [25]:
from tqdm.notebook import tqdm

def build_tag_dict(tags, tag_categories):
    tag_dict = {}
    for tag, tag_category in zip(tags, tag_categories):
        if tag_category not in tag_dict:
            tag_dict[tag_category] = []
        tag_dict[tag_category].append(tag)
    return tag_dict

tqdm.pandas()

outfits_df["tag_dict"] = outfits_df.progress_apply(lambda x: build_tag_dict(x["outfit_tags"], x["tag_categories"]), axis=1)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

all_tags = outfits_df["outfit_tags"].values.tolist()
mlb = MultiLabelBinarizer()
one_hot_encoded = mlb.fit_transform(all_tags)

NameError: name 'tags_list' is not defined

In [26]:
outfits_df["tag_dict"]

,id,name,description,group,owner,timeCreated,retailPrice,outfit_tags,tag_categories,embeddings,mean_embeddings,tag_dict
0,outfit.fffdaa715c3646f8b1c0f04d549ff07e,Out of stock - Asymmetric Frilled Dress,"This fun, short dress features and asymmetric ...",group.50a586c78eb7626e294ba3bd07d12c79,464,2017-12-30 11:28:01.000,4000.0,"[Metallic, Synthetic, Cotton, Sandro, Dresses,...","[Details, Material, Material, Brand, Category,...","[[1.738, -0.0944, -0.0934, 0.1979, 0.2365, -0....","[0.7656, -0.073, -0.0672, 0.1146, 0.503, -0.03...","{'Details': ['Metallic'], 'Material': ['Synthe..."
1,outfit.fffa1b9a3db6415d806f3c48f8ab58d9,Yellow Shell Mellomholmene Blouse,This beautiful blouse features an adjustable n...,group.61ad2fcabb3e9197e3836376e6b67f2c,112,2021-06-07 12:07:22.921,1300.0,"[Yellow, Cotton, Blouses, Everyday, M, Summer,...","[Color, Material, Category, Occasion, Size, Se...","[[-0.0843, -0.0567, -0.05966, -0.077, 1.166, -...","[-0.0943, -0.0998, -0.003277, -0.1155, 0.378, ...","{'Color': ['Yellow'], 'Material': ['Cotton'], ..."
2,outfit.fff175b13ceb453f9928625491412ede,Kaula Dress Black,Kaula from Rodebjer is a fitted dress made in ...,group.37c2b59d63d3a9c2d58e07f532f71f7f,635,2023-06-05 09:17:59.004,3100.0,"[Synthetic, Multi Season, Rodebjer, Everyday, ...","[Material, Seasons, Brand, Occasion, Size, Cat...","[[1.27, -0.0494, -0.02313, -0.1021, 0.2625, 0....","[0.4563, -0.05505, -0.03162, -0.01764, 1.111, ...","{'Material': ['Synthetic'], 'Seasons': ['Multi..."
3,outfit.ffef9d7c292a48b69076d2df2e32352f,For sale - Jarvis Blouse,This wrap blouse has mid length sleeves and a ...,group.dfcaa57546b0b7a5e9eb204449b6cc1c,745,2021-05-18 14:02:28.690,1500.0,"[Cotton, Multi Season, Floral, Wrap, XS, Style...","[Material, Seasons, Details, Fit, Size, Brand,...","[[-0.04453, -0.08777, -0.0676, -0.07196, 0.086...","[-0.064, -0.0856, -0.0883, -0.07697, -0.02661,...","{'Material': ['Cotton'], 'Seasons': ['Multi Se..."
4,outfit.ffeef842238f4dbdabc6c730a75aa2bd,Black Amber Pants,"Feel slack and nice dressed with this pant, ma...",group.ee297c977905eb21a123a4aea5fbb6d2,504,2021-07-16 14:02:30.643,1200.0,"[Winter, Cotton, L, Knitwear, Everyday, Fall, ...","[Seasons, Material, Size, Category, Occasion, ...","[[0.02425, -0.1558, -0.1343, -0.07513, -0.0170...","[-0.03262, -0.04355, -0.0886, -0.0684, -0.0862...","{'Seasons': ['Winter', 'Fall'], 'Material': ['..."
...,...,...,...,...,...,...,...,...,...,...,...,...
15824,outfit.001bf665330140cf854dcfb1cbff6b5f,Out of stock - Harley Vintage White Midi Dress,This gorgeous dress is cut in the most flatter...,group.d91a2a6728833c8082dadf27b95488a9,140,2019-06-25 10:13:55.000,3800.0,"[Viscose, L, Midi, Dresses, White, Formal, Pia...","[Material, Size, Length, Category, Color, Occa...","[[-0.10986, -0.05212, -0.04785, -0.1338, 0.035...","[0.5796, 0.084, 0.866, 0.1619, 0.926, -0.02487...","{'Material': ['Viscose'], 'Size': ['L'], 'Leng..."
15825,outfit.0018701ce6b049ebadc314d16623caa8,Vintage Burberry Trench Coat,You really can't go wrong with this Classic Tr...,group.6be510229d0f9faf5d19d52e7e2b2a95,58,2023-02-07 07:54:06.214,22000.0,"[Winter, Cotton, Midi, Everyday, Fall, Burberr...","[Seasons, Material, Length, Occasion, Seasons,...","[[0.9565, 0.6475, -0.0587, 0.704, 0.2399, 0.04...","[0.838, 0.1426, -0.0905, 0.456, 0.0379, -0.068...","{'Seasons': ['Winter', 'Fall'], 'Material': ['..."
15826,outfit.0014a5c89b244077a3d7cffd4549718e,Mira Skirt Brown,The Mira Skirt in Brown from Stine Goya is an ...,group.668be5db7976aa2cb9213dd4c7f9b7fe,4,2023-10-09 09:12:14.631,1500.0,"[Viscose, Midi, Skirts, Summer, Stine Goya, Ev...","[Material, Length, Category, Seasons, Brand, B...","[[-0.1237, -0.03632, -0.08435, -0.1036, 1.478,...","[-0.0696, -0.0561, -0.051, -0.0895, 1.148, 0.1...","{'Material': ['Viscose'], 'Length': ['Midi'], ..."
15827,outfit.0013691ff35b440e9dcfe1748ec184c7,Oldina Parka Cotta,The Oldina Parka from Kari Traa is a women's p...,group.c82046bcba672c8ec9b21be4f844b402,552,2023-02-23 12:20:27.

In [16]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
from tqdm.notebook import tqdm

NUM_ITEMS = 100

def find_rental_history_embeddings(outfit_ids, outfit_to_embedding_dict):
    return [outfit_to_embedding_dict[outfit_id] for outfit_id in outfit_ids]

def get_mean_embedding(embeddings):
    embeddings = np.array(embeddings)
    return np.mean(embeddings, axis=0)

def get_nearest_neighbors_batch(embeddings, nn, num_items, index_to_id):
    distances, indices = nn.kneighbors(embeddings, n_neighbors=num_items+1)
    ids = [[index_to_id[i] for i in idx[1:]] for idx in indices]
    distances = [dist[1:] for dist in distances]
    return ids, distances

def predict_nearest_neighbors(df, outfits_df, subset_length=-1):
    outfits_df["mean_embeddings"] = outfits_df["embeddings"].apply(lambda x: get_mean_embedding(x))
    outfit_to_embedding_dict = outfits_df.set_index("id")["mean_embeddings"].to_dict()
    index_to_outfit_dict = {i: outfit_id for i, outfit_id in enumerate(outfits_df["id"].values)}
    group_to_embedding_dict = outfits_df.set_index("group")["mean_embeddings"].to_dict()
    index_to_group_dict = {i: group for i, group in enumerate(outfits_df["group"].values)}

    df["train_id_embeddings"] = df["train_outfit_ids"].apply(lambda x: find_rental_history_embeddings(x, outfit_to_embedding_dict))
    df["train_group_embeddings"] = df["train_group"].apply(lambda x: find_rental_history_embeddings(x, group_to_embedding_dict))

    df["rental_history_id_embedding"] = df["train_id_embeddings"].apply(lambda x: get_mean_embedding(x))
    df["rental_history_group_embedding"] = df["train_group_embeddings"].apply(lambda x: get_mean_embedding(x))

    nearest_neighbors = NearestNeighbors(n_neighbors=NUM_ITEMS+1, metric="cosine")
    embeddings = np.stack(outfits_df["mean_embeddings"].values)
    nearest_neighbors.fit(embeddings)

    id_embeddings = np.stack(df["rental_history_id_embedding"].values)
    group_embeddings = np.stack(df["rental_history_group_embedding"].values)

    id_predictions, id_distances = get_nearest_neighbors_batch(id_embeddings, nearest_neighbors, NUM_ITEMS, index_to_outfit_dict)
    group_predictions, group_distances = get_nearest_neighbors_batch(group_embeddings, nearest_neighbors, NUM_ITEMS, index_to_group_dict)

    df["id_prediction"], df["id_prediction_distances"] = id_predictions, id_distances
    df["group_prediction"], df["group_prediction_distances"] = group_predictions, group_distances
    
    return df

# Apply to dataframes
tqdm.pandas()
user_splits_df = predict_nearest_neighbors(user_splits_df, outfits_df, subset_length=-1)
user_splits_unique_df = predict_nearest_neighbors(user_splits_unique_df, outfits_df, subset_length=-1)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Perform t-SNE
tsne = TSNE(n_components=2, random_state=42)
embeddings_tsne = tsne.fit_transform(embeddings)

# Plot the t-SNE result
plt.figure(figsize=(10, 8))
plt.scatter(embeddings_tsne[:, 0], embeddings_tsne[:, 1], s=5, cmap='viridis')
plt.title('t-SNE of Embeddings')
plt.xlabel('t-SNE component 1')
plt.ylabel('t-SNE component 2')
plt.show()

In [63]:
# Test if test outfit is in train outfits for recommendation of repeated outfits
# def check_if_test_in_train(train_outfit_ids, test_outfit_id):
#     return test_outfit_id in train_outfit_ids

# user_splits_df["test_in_train"] = user_splits_df.apply(lambda x: check_if_test_in_train(x["train_outfit_ids"], x["test_outfit_id"]), axis=1)
# user_splits_unique_df["test_in_train"] = user_splits_unique_df.apply(lambda x: check_if_test_in_train(x["train_outfit_ids"], x["test_outfit_id"]), axis=1)
# display(user_splits_df["test_in_train"].value_counts())
# user_splits_df[["train_outfit_ids", "test_outfit_id", "id_prediction", "test_in_train"]]
# display(user_splits_unique_df["test_in_train"].value_counts())
# user_splits_unique_df[["train_outfit_ids", "test_outfit_id", "id_prediction", "test_in_train"]]


# user_splits_unique_df["train_length"] = user_splits_unique_df["train_outfit_ids"].apply(len)
# user_splits_unique_df["prediction_length"] = user_splits_unique_df["id_prediction"].apply(len)

In [22]:
from IPython.display import display

def evaluate_hit_rate_at_n(test_id, predicted_ids, n=10):
    if predicted_ids is np.nan:
        print(f"None prediction for {test_id}!")
        return 0
    predicted_ids = predicted_ids[:n]
    if test_id in predicted_ids:
        #print(f"Hit at {n} for {test_id} in {predicted_ids}")
        return 1
    return 0

HIT_RATE_COLUMNS = ["id_hit_rate_at_100", "id_hit_rate_at_10", "group_hit_rate_at_100", "group_hit_rate_at_10"]
def evaluate_df_hit_rate_at_n(df, n=10):
    df["id_hit_rate_at_100"] = df.apply(lambda x: evaluate_hit_rate_at_n(x["test_outfit_id"], x["id_prediction"], n=100), axis=1)
    df["id_hit_rate_at_10"] = df.apply(lambda x: evaluate_hit_rate_at_n(x["test_outfit_id"], x["id_prediction"], n=10), axis=1)
    df["group_hit_rate_at_100"] = df.apply(lambda x: evaluate_hit_rate_at_n(x["test_group"], x["group_prediction"], n=100), axis=1)
    df["group_hit_rate_at_10"] = df.apply(lambda x: evaluate_hit_rate_at_n(x["test_group"], x["group_prediction"], n=10), axis=1)
    display(df[HIT_RATE_COLUMNS].mean())
    return df


user_splits_df = evaluate_df_hit_rate_at_n(user_splits_df, n=10)
user_splits_unique_df = evaluate_df_hit_rate_at_n(user_splits_unique_df, n=10)

id_hit_rate_at_100       0.043321
id_hit_rate_at_10        0.009477
group_hit_rate_at_100    0.047383
group_hit_rate_at_10     0.012184
dtype: float64

id_hit_rate_at_100       0.044695
id_hit_rate_at_10        0.007223
group_hit_rate_at_100    0.047856
group_hit_rate_at_10     0.009481
dtype: float64

In [23]:
import pyperclip

def format_dicts_into_latex(all_dict, ind_dict, precision=4, run_name="Random"):
    first_row = f"{run_name} Ind: & {all_dict['id_hit_rate_at_10']:.{precision}f} & {all_dict['id_hit_rate_at_100']:.{precision}f} & {ind_dict['id_hit_rate_at_10']:.{precision}f} & {ind_dict['id_hit_rate_at_100']:.{precision}f} \\\\"
    second_row = f"{run_name} Groups: & {all_dict['group_hit_rate_at_10']:.{precision}f} & {all_dict['group_hit_rate_at_100']:.{precision}f} & {ind_dict['group_hit_rate_at_10']:.{precision}f} & {ind_dict['group_hit_rate_at_100']:.{precision}f} \\\\\\hline"
    full_string = first_row + "\n" + second_row + "\n"
    print(full_string)
    pyperclip.copy(full_string)

all_dict = {column: user_splits_df[column].mean() for column in HIT_RATE_COLUMNS}
ind_dict = {column: user_splits_unique_df[column].mean() for column in HIT_RATE_COLUMNS}

format_dicts_into_latex(all_dict, ind_dict, precision=4, run_name="Image Embeddings")

Image Embeddings Ind: & 0.0095 & 0.0433 & 0.0072 & 0.0447 \\
Image Embeddings Groups: & 0.0122 & 0.0474 & 0.0095 & 0.0479 \\\hline



In [109]:
import numpy as np

def get_outfit_category(tag_categories, tags, category):
    tag_categories, tags = np.array(tag_categories), np.array(tags)
    category_indexes = np.where(tag_categories == category)[0]
    if len(category_indexes) == 0:
        return ""
    cat_tags = tags[category_indexes]
    output = str(cat_tags[0])
    return output

outfits_df["size"] = outfits_df.apply(lambda x: get_outfit_category(x["tag_categories"], x["outfit_tags"], "Size"), axis=1)


In [78]:
outfits_df

,id,name,description,group,owner,timeCreated,retailPrice,outfit_tags,tag_categories,size
0,outfit.fffdaa715c3646f8b1c0f04d549ff07e,Out of stock - Asymmetric Frilled Dress,"This fun, short dress features and asymmetric ...",group.50a586c78eb7626e294ba3bd07d12c79,464,2017-12-30 11:28:01.000,4000.0,"[Metallic, Synthetic, Cotton, Sandro, Dresses,...","[Details, Material, Material, Brand, Category,...",S
1,outfit.fffa1b9a3db6415d806f3c48f8ab58d9,Yellow Shell Mellomholmene Blouse,This beautiful blouse features an adjustable n...,group.61ad2fcabb3e9197e3836376e6b67f2c,112,2021-06-07 12:07:22.921,1300.0,"[Yellow, Cotton, Blouses, Everyday, M, Summer,...","[Color, Material, Category, Occasion, Size, Se...",M
2,outfit.fff175b13ceb453f9928625491412ede,Kaula Dress Black,Kaula from Rodebjer is a fitted dress made in ...,group.37c2b59d63d3a9c2d58e07f532f71f7f,635,2023-06-05 09:17:59.004,3100.0,"[Synthetic, Multi Season, Rodebjer, Everyday, ...","[Material, Seasons, Brand, Occasion, Size, Cat...",M
3,outfit.ffef9d7c292a48b69076d2df2e32352f,For sale - Jarvis Blouse,This wrap blouse has mid length sleeves and a ...,group.dfcaa57546b0b7a5e9eb204449b6cc1c,745,2021-05-18 14:02:28.690,1500.0,"[Cotton, Multi Season, Floral, Wrap, XS, Style...","[Material, Seasons, Details, Fit, Size, Brand,...",XS
4,outfit.ffeef842238f4dbdabc6c730a75aa2bd,Black Amber Pants,"Feel slack and nice dressed with this pant, ma...",group.ee297c977905eb21a123a4aea5fbb6d2,504,2021-07-16 14:02:30.643,1200.0,"[Winter, Cotton, L, Knitwear, Everyday, Fall, ...","[Seasons, Material, Size, Category, Occasion, ...",L
...,...,...,...,...,...,...,...,...,...,...
15824,outfit.001bf665330140cf854dcfb1cbff6b5f,Out of stock - Harley Vintage White Midi Dress,This gorgeous dress is cut in the most flatter...,group.d91a2a6728833c8082dadf27b95488a9,140,2019-06-25 10:13:55.000,3800.0,"[Viscose, L, Midi, Dresses, White, Formal, Pia...","[Material, Size, Length, Category, Color, Occa...",L
15825,outfit.0018701ce6b049ebadc314d16623caa8,Vintage Burberry Trench Coat,You really can't go wrong with this Classic Tr...,group.6be510229d0f9faf5d19d52e7e2b2a95,58,2023-02-07 07:54:06.214,22000.0,"[Winter, Cotton, Midi, Everyday, Fall, Burberr...","[Seasons, Material, Length, Occasion, Seasons,...",S
15826,outfit.0014a5c89b244077a3d7cffd4549718e,Mira Skirt Brown,The Mira Skirt in Brown from Stine Goya is an ...,group.668be5db7976aa2cb9213dd4c7f9b7fe,4,2023-10-09 09:12:14.631,1500.0,"[Viscose, Midi, Skirts, Summer, Stine Goya, Ev...","[Material, Length, Category, Seasons, Brand, B...",S
15827,outfit.0013691ff35b440e9dcfe1748ec184c7,Oldina Parka Cotta,The Oldina Parka from Kari Traa is a women's p...,group.c82046bcba672c8ec9b21be4f844b402,552,2023-02-23 12:20:27.042,3500.0,"[Winter, Synthetic, Midi, Everyday, XS, Coats,...","[Seasons, Material, Length, Occasion, Size, Ca...",XS
